In [1]:
from pathlib import Path
import pandas as pd
import string


In [2]:
df = pd.read_csv("010_some_models_transcriptions.csv")
df.head()

,model,transcription
0,classla/wav2vec2-xls-r-parlaspeech-hr-lm,prije zapinfacijma ćearavaaženv hvala predsjed...
1,classla/wav2vec2-xls-r-parlaspeech-hr,oe prig zaopinforacijma zk paajaćeapravae zap...
2,classla/wav2vec2-large-slavic-parlaspeech-hr,švavatenu tražim u ime kluba stranke seta st...
3,classla/wav2vec2-large-slavic-parlaspeech-hr-lm,švvatenutražm u ime kluba stranke set tanka gl...


In [3]:
def generate_differ_content(pred:str) -> str:
    """Generate comparatively pretty diff from input predictions
    against global raw.

    Args:
        pred (str): Predictions to be compared

    Returns:
        str: Newline delimited diff.
    """
    from difflib import Differ
    differ_content = ""
    for diff in list(Differ().compare(raw.split(), pred.split()))[:500]:
        differ_content += diff + "\n"
    return differ_content

In [8]:
wers = []
cers = []
for i, row in df.iterrows():
    modelname = row["model"]
    raw = df.iloc[1,1]
    pred = row["transcription"].replace("<pad>", "")
    from datasets import load_metric

    wer = load_metric("wer")
    cer = load_metric("cer")

    c = cer.compute(predictions = [pred], references = [raw])
    w = wer.compute(predictions = [pred], references = [raw])
    cers.append(c)
    wers.append(w)
    content = generate_differ_content(pred=pred)



In [9]:
df["cer"] = cers
df["wer"] = wers
print(df["model wer cer".split()].to_markdown(index=False))

| model                                           |      wer |       cer |
|:------------------------------------------------|---------:|----------:|
| classla/wav2vec2-xls-r-parlaspeech-hr-lm        | 0.144981 | 0.0387541 |
| classla/wav2vec2-xls-r-parlaspeech-hr           | 0        | 0         |
| classla/wav2vec2-large-slavic-parlaspeech-hr    | 0.329926 | 0.117841  |
| classla/wav2vec2-large-slavic-parlaspeech-hr-lm | 0.282528 | 0.119994  |
